In [1]:
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import numpy as np
import cv2
import time
from skimage.feature import hog
from sklearn.svm import LinearSVC
from sklearn.preprocessing import StandardScaler
from sklearn.cross_validation import train_test_split
import os
import glob
import pickle
from moviepy.editor import *
from scipy.ndimage.measurements import label
# assigning a random seed so that the results are reproducible
np.random.seed(42)
%matplotlib inline

In [2]:
# Define a function to return HOG features and visualization
def get_hog_features(img, orient, pix_per_cell, cell_per_block, vis=False, feature_vec=True):
    if vis == True:
        features, hog_image = hog(img, orientations=orient, 
                                  pixels_per_cell=(pix_per_cell, pix_per_cell),
                                  cells_per_block=(cell_per_block, cell_per_block), 
                                  transform_sqrt=False, visualise=True, feature_vector=feature_vec)
        return features, hog_image
    else:      
        features = hog(img, orientations=orient, 
                       pixels_per_cell=(pix_per_cell, pix_per_cell),
                       cells_per_block=(cell_per_block, cell_per_block),
                       transform_sqrt=False, visualise=False, feature_vector=feature_vec)
        return features

# Define a function to compute binned color features
def bin_spatial(img, size=(32, 32)):
    # Use cv2.resize().ravel() to create the feature vector
    color1 = cv2.resize(img[:, :, 0], size).ravel()
    color2 = cv2.resize(img[:, :, 1], size).ravel()
    color3 = cv2.resize(img[:, :, 2], size).ravel()
    # Return the feature vector
    return np.hstack((color1, color2, color3))

# Define a function to compute color histogram features  
def color_hist(img, nbins=32):
    # Compute the histogram of the color channels separately
    channel1_hist = np.histogram(img[:,:,0], bins=nbins)
    channel2_hist = np.histogram(img[:,:,1], bins=nbins)
    channel3_hist = np.histogram(img[:,:,2], bins=nbins)
    # Concatenate the histograms into a single feature vector
    hist_features = np.concatenate((channel1_hist[0], channel2_hist[0], channel3_hist[0]))
    # Return the individual histograms, bin_centers and feature vector
    return hist_features

In [3]:
# Define a function to extract features from a single image window
# This function is very similar to extract_features()
# just for a single image rather than list of images
def single_img_features(img, color_space='YCrCb', orient=9, pix_per_cell=8, cell_per_block=2, 
                        hog_channel=0, spatial_size=(32, 32), hist_bins=32, spatial_feat=True,
                        hist_feat=True, hog_feat=True, vis=False, feature_vec=True):
    
    # Define an empty list to receive features
    img_features = []
    # Apply color conversion if other than 'RGB'
    if color_space != 'RGB':
        if color_space == 'HSV':
            feature_image = cv2.cvtColor(img, cv2.COLOR_RGB2HSV)
        elif color_space == 'YCrCb':
            feature_image = cv2.cvtColor(img, cv2.COLOR_RGB2YCrCb)
    else:
        feature_image = np.copy(img)
    # Compute spatial features if flag is set
    if spatial_feat:
        spatial_features = bin_spatial(feature_image, size=spatial_size)
        # Append features to list
        img_features.append(spatial_features)
    # Compute histogram features if flag is set
    if hist_feat:
        hist_features = color_hist(feature_image, nbins=hist_bins)
        # Append features to list
        img_features.append(hist_features)
    # Compute HOG features if flag is set
    if hog_feat:
        if hog_channel == 'ALL':
            hog_features = []
            for channel in range(feature_image.shape[2]):
                hog_features.append(get_hog_features(feature_image[:,:,channel],
                                    orient, pix_per_cell, cell_per_block,
                                    vis=False, feature_vec=True))
            hog_features = np.concatenate(hog_features)
        else:
            if vis == True:
                hog_features, hog_image = get_hog_features(feature_image[:, :, hog_channel],
                                                orient, pix_per_cell,
                                                cell_per_block, vis=True,
                                                feature_vec=True)
            else:
                hog_features = get_hog_features(feature_image[:, :, hog_channel],
                                                orient, pix_per_cell,
                                                cell_per_block, vis=False,
                                                feature_vec=True)
        # Append features to list
        img_features.append(hog_features)
    if vis == True:
        return np.concatenate(img_features), hog_image
    else:
        # Return concatenated array of features
        return np.concatenate(img_features)

In [4]:
# Define a function that takes an image,
# start and stop positions in both x and y,
# window size (x and y dimensions),
# and overlap fraction (for both x and y)
def slide_window(img, x_start_stop=[None, None], y_start_stop=[None, None],
                 xy_window=(64, 64), xy_overlap=(0.5, 0.5)):
    # If x and/or y start/stop positions not defined, set to image size
    if x_start_stop[0] is None:
        x_start_stop[0] = 0
    if x_start_stop[1] is None:
        x_start_stop[1] = img.shape[1]
    if y_start_stop[0] is None:
        y_start_stop[0] = 0
    if y_start_stop[1] is None:
        y_start_stop[1] = img.shape[0]

    # Compute the span of the region to be searched
    xspan = x_start_stop[1] - x_start_stop[0]
    yspan = y_start_stop[1] - y_start_stop[0]

    # Compute the number of pixels per step in x/y
    nx_pix_per_step = np.int(xy_window[0]*(1 - xy_overlap[0]))
    ny_pix_per_step = np.int(xy_window[1]*(1 - xy_overlap[1]))

    # Compute the number of windows in x/y
    nx_buffer = np.int(xy_window[0]*(xy_overlap[0]))
    ny_buffer = np.int(xy_window[1]*(xy_overlap[1]))
    nx_windows = np.int((xspan-nx_buffer)/nx_pix_per_step)
    ny_windows = np.int((yspan-ny_buffer)/ny_pix_per_step)

    # Initialize a list to append window positions to
    window_list = []
    # Loop through finding x and y window positions
    for ys in range(ny_windows):
        for xs in range(nx_windows):
            # Calculate window position
            startx = xs*nx_pix_per_step + x_start_stop[0]
            endx = startx + xy_window[0]
            starty = ys*ny_pix_per_step + y_start_stop[0]
            endy = starty + xy_window[1]
            # Append window position to list
            window_list.append(((startx, starty), (endx, endy)))
    # Return the list of windows
    return window_list


# Define a function you will pass an image
# and the list of windows to be searched (output of slide_windows())
def search_windows(img, windows, clf, scaler, color_space, orient, pix_per_cell, cell_per_block, 
                   hog_channel, spatial_size, hist_bins, spatial_feat, hist_feat, hog_feat):
    
    # 1) Create an empty list to receive positive detection windows
    on_windows = []
    # 2) Iterate over all windows in the list
    for window in windows:
        # since our training data is of size (64, 64), we resize the window to these dimensions
        # in case we choose windows of different sizes, we always want to extract features based
        # on what we trained on
        # -> do not change these values according to the window size
        test_img = cv2.resize(img[window[0][1]:window[1][1], window[0][0]:window[1][0]], (64, 64))

        # Extract features for that window using single_img_features()
        features = single_img_features(test_img, color_space, orient, pix_per_cell, cell_per_block, 
                                       hog_channel, spatial_size, hist_bins, spatial_feat, hist_feat, hog_feat)

        # Scale extracted features to be fed to classifier
        test_features = scaler.transform(np.array(features).reshape(1, -1))
        # 6) Predict using your classifier
        prediction = clf.predict(test_features)
        # 7) If positive (prediction == 1) then save the window
        if prediction == 1:
            on_windows.append(window)
    # 8) Return windows for positive detections
    return on_windows

In [104]:
def visualize(fig, rows, cols, images):
    for i, img in enumerate(images):
        plt.subplot(rows, cols, i+1)
        img_dims = len(img.shape)
        if img_dims < 3:
            plt.imshow(img, cmap='hot')
        else:
            plt.imshow(img)
            
# Define a function to draw bounding boxes
def draw_boxes(img, bboxes, color=(0, 0, 255), thick=6):
    # Make a copy of the image
    imcopy = np.copy(img)
    # Iterate through the bounding boxes
    for bbox in bboxes:
        # Draw a rectangle given bbox coordinates
        cv2.rectangle(imcopy, bbox[0], bbox[1], color, thick)
    # Return the image copy with boxes drawn
    return imcopy

def read_models(color_space):
    with open(color_space + "_linear_svm_best.pkl", "rb") as clf_pickle:
        clf = pickle.load(clf_pickle)
    with open(color_space + "_scaling_model.pkl", "rb") as scalar_pickle:
        scalar = pickle.load(scalar_pickle)
    return clf, scalar

def draw_labeled_bboxes(img, labels):
    # Iterate through all detected cars
    for car_number in range(1, labels[1]+1):
        # Find pixels with each car_number label value
        nonzero = (labels[0] == car_number).nonzero()
        # Identify x and y values of those pixels
        nonzeroy = np.array(nonzero[0])
        nonzerox = np.array(nonzero[1])
        # Define a bounding box based on min/max x and y
        bbox = ((np.min(nonzerox), np.min(nonzeroy)), (np.max(nonzerox), np.max(nonzeroy)))
        if (len(nonzeroy) * len(nonzerox) > 200):
            # Draw the box on the image
            cv2.rectangle(img, bbox[0], bbox[1], (0,0,255), 6)
    # Return the image
    return img

In [105]:
# Add heat to heatmap
def add_heat(heatmap, bbox_list):
    # Iterate through list of bboxes
    for box in bbox_list:
        # Add += 1 for all pixels inside each bbox
        #  Assuming each "box" takes the form ((x1, y1), (x2, y2))
        heatmap[box[0][1]:box[1][1], box[0][0]:box[1][0]] += 1

    # Return updated heatmap
    return heatmap


# Apply threshold to heat map
def apply_threshold(heatmap, threshold):
    # Zero out pixels below the threshold
    heatmap[heatmap <= threshold] = 0
    # Return thresholded map
    return heatmap


In [121]:
def pipeline(img):
    global all_windows
    
    color_space = 'YCrCb'
    clf, scalar = read_models(color_space)
#     img = plt.imread(img)
    draw_img = np.copy(img)
    on_image = np.copy(img)
    img = img.astype(np.float32)/255
    y_start_stop = [400, 600]
    # get all the windows depending on windows size and overlap
    t = time.time()
    windows = slide_window(img, 
                       x_start_stop=[200, 1200], 
                       y_start_stop=y_start_stop, 
                       xy_window=(96, 96),
                       xy_overlap=(0.7, 0.7))
    # make predictions on every window and make a list of windows that detected the presence of a car
    hot_windows = search_windows(img, windows, clf, scalar, color_space=color_space, orient=9, pix_per_cell=8,
                                 cell_per_block=2, hog_channel='ALL', spatial_size=(32, 32), hist_bins=32,
                                 spatial_feat=True, hist_feat=True, hog_feat=True)

    if len(all_windows) > 60:
#         _ = all_windows.pop(0)
        del all_windows[:-20]
        all_windows += hot_windows
    else:
        all_windows += hot_windows
        
    t2 = time.time()
    print(round(t2 - t, 5), 'Seconds to predict using SVC...')

    window_img = draw_boxes(draw_img, hot_windows, color=(0, 0, 255), thick=6)
    # Calculate and draw heat map
    heatmap = np.zeros((720, 1280))
    heatmap = add_heat(heatmap, all_windows)
    heatmap = apply_threshold(heatmap, 8)
    labels = label(heatmap)
    # Draw final bounding boxes
    draw_img = draw_labeled_bboxes(on_image, labels)

    return draw_img

In [122]:
all_windows = []
test_output = 'project_output_final.mp4'
clip1 = VideoFileClip('project_video.mp4')
test_clip = clip1.fl_image(pipeline)
test_clip.write_videofile(test_output, audio=False)

0.98401 Seconds to predict using SVC...
[MoviePy] >>>> Building video project_output_final.mp4
[MoviePy] Writing video project_output_final.mp4






  0%|          | 0/1261 [00:00<?, ?it/s]



  0%|          | 1/1261 [00:00<18:37,  1.13it/s]

0.8599 Seconds to predict using SVC...






  0%|          | 2/1261 [00:01<18:48,  1.12it/s]

0.87577 Seconds to predict using SVC...


...





100%|█████████▉| 1259/1261 [20:29<00:01,  1.01it/s]

0.90633 Seconds to predict using SVC...






100%|█████████▉| 1260/1261 [20:30<00:00,  1.02it/s]

0.90149 Seconds to predict using SVC...


[MoviePy] Done.
[MoviePy] >>>> Video ready: project_output_final.mp4 

